
<h1>Power System Analysis and Optimization (5XWA0) - Practice</h1>
<h2>Economic dispatch with generation constraints in PYOMO</h2>

<b>MSc Irena Dukovska</b> <br />
<b>Dr. Nikolaos Paterakis</b> <br />
  Electrical Energy Systems Group <br />
  Department of Electrical Engineering <br />
  Eindhoven University of Technology <br />
  (i.dukovska@tue.nl, n.paterakis@tue.nl) <br /><br />


<h2>Introduction</h2>

The optimization problem is formulated as follows:

\begin{align}
&\text{Minimize} \ C = \sum_{i}(a_i+b_i \cdot P_i + c_i \cdot P_i^2) \\
&\text{subject to:} \\
&\sum_{i} P_i = D \ \forall i \\
&P_i^{min} \leq P_i \leq P_i^{max} \ \forall i \\
&P_i \geq 0 \ \forall i
\end{align}

The goal of the optimization problem is to find the optimal power output of each unit in order to serve the total load at a minimum cost.

<b>In this tutorial we will:</b> <br>
1) develop an optimization model using Pyomo to solve the problem above <br>
2) show how to extract the results of the solved model <br>
3) visualize the results using Matplotlib library

In [86]:
import pandas
import numpy
from pyomo.environ import *

model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

In [87]:
UnitData = pandas.read_excel('ED_input.xlsx', sheet_name = 'Units', index_col= 0)
D = 550
print(UnitData)

         a    b      c  Min  Max
                                
Unit1  500  5.3  0.004  200  450
Unit2  400  5.5  0.006  150  350
Unit3  200  5.8  0.009  100  225


In [88]:
model.I = Set(ordered=True, initialize=UnitData.index)

model.Pmax = Param(model.I, within = NonNegativeReals, mutable = True)
model.Pmin = Param(model.I, within = NonNegativeReals, mutable = True)

model.a = Param(model.I, within = NonNegativeReals, mutable = True)
model.b = Param(model.I, within = NonNegativeReals, mutable = True)
model.c = Param(model.I, within = NonNegativeReals, mutable = True)

for i in model.I:
    model.Pmax[i] = UnitData.loc[i,'Max']
    model.Pmin[i] = UnitData.loc[i,'Min']
    model.a[i] = UnitData.loc[i,'a']
    model.b[i] = UnitData.loc[i,'b']
    model.c[i] = UnitData.loc[i,'c']

model.P = Var(model.I, within = PositiveReals)

In [89]:
def obj_rule(model):
    return sum(model.a[i] + model.b[i]*model.P[i] + model.c[i]*model.P[i]*model.P[i] for i in model.I)

def min_rule(model, i):
    return model.P[i] >= model.Pmin[i]

def max_rule(model, i):
    return model.P[i] <= model.Pmax[i]

def pbalance_rule(model):
    return sum(model.P[i] for i in model.I) == D

In [90]:
model.cost = Objective(rule = cost_rule)
model.unit_out_min_constraint = Constraint(model.I, rule = min_rule)
model.unit_out_max_constraint = Constraint(model.I, rule = max_rule)
model.balance = Constraint(rule = pbalance_rule)

opt=SolverFactory('gurobi')
results=opt.solve(model)

Unit1
a[Unit1]
b[Unit1]*P[Unit1]
Unit2
a[Unit2]
b[Unit2]*P[Unit2]
Unit3
a[Unit3]
b[Unit3]*P[Unit3]


In [91]:
model
#print the marginal value of the power balance constraint
print("Marginal cost: ",model.balance.get_suffix_value(model.dual))

#print the total production cost
print("Total cost: ", model.cost())

Marginal cost:  7.5399999999719425
Total cost:  4676.00000000035
